In [1]:
from harpy import (
    Session,
    MapTask,
    ReduceTask,
    TransformTask,
    Result,
    TaskSetResults
)

session = Session().create_session()

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Socket closed"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"Socket closed", grpc_status:14, created_time:"2024-10-06T12:56:57.026414994+00:00"}"
>

In [2]:
path = "/home/ghhwer/project-quack/backend/client-engine/_example_data/Flights_1m.parquet"
result = session.sql(f"SELECT * FROM read_parquet('{path}')")

TaskSet ts-0b6ce951-085c-4b11-b73d-a9632f254fbc: running
TaskGroup tg-7520d4ca-a5eb-41a8-858c-92d92cd08046 made progress
TaskGroup tg-7520d4ca-a5eb-41a8-858c-92d92cd08046 made progress
Task tg-7520d4ca-a5eb-41a8-858c-92d92cd08046-tr-0: queued
TaskGroup tg-7520d4ca-a5eb-41a8-858c-92d92cd08046 made progress
Task tg-7520d4ca-a5eb-41a8-858c-92d92cd08046-tr-0: running
Task tg-7520d4ca-a5eb-41a8-858c-92d92cd08046-tr-0: done
TaskGroup tg-7520d4ca-a5eb-41a8-858c-92d92cd08046 made progress
TaskSet ts-0b6ce951-085c-4b11-b73d-a9632f254fbc: compleated
Getting results


In [3]:
result

,FL_DATE,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE,DEP_TIME,ARR_TIME
0,2006-01-01,5,19,350,2475,9.083333,12.483334
1,2006-01-02,167,216,343,2475,11.783334,15.766666
2,2006-01-03,-7,-2,344,2475,8.883333,12.133333
3,2006-01-04,-5,-13,331,2475,8.916667,11.950000
4,2006-01-05,-3,-17,321,2475,8.950000,11.883333
...,...,...,...,...,...,...,...
999995,2006-02-20,87,86,71,545,19.733334,21.333334
999996,2006-02-20,-2,0,71,508,8.050000,9.583333
999997,2006-02-20,33,19,64,508,14.700000,15.983334
999998,2006-02-20,65,54,65,508,17.633333,18.983334


In [4]:
result = session.sql(f"SELECT max(FL_DATE) FROM read_parquet('{path}')")

TaskSet ts-fed9e032-562e-460b-8319-7f2d54223c74: running
TaskGroup tg-5198e9bc-9b9a-40d7-a0df-92cca6f0289e made progress
TaskGroup tg-5198e9bc-9b9a-40d7-a0df-92cca6f0289e made progress
Task tg-5198e9bc-9b9a-40d7-a0df-92cca6f0289e-tr-0: queued
TaskGroup tg-5198e9bc-9b9a-40d7-a0df-92cca6f0289e made progress
Task tg-5198e9bc-9b9a-40d7-a0df-92cca6f0289e-tr-0: running
Task tg-5198e9bc-9b9a-40d7-a0df-92cca6f0289e-tr-0: done
TaskGroup tg-5198e9bc-9b9a-40d7-a0df-92cca6f0289e made progress
TaskSet ts-fed9e032-562e-460b-8319-7f2d54223c74: compleated
Getting results


In [5]:
result

,max(FL_DATE)
0,2006-02-28


In [6]:
session.close()